In [ ]:
sos run xqtl-pipeline/code/misc/build_container.ipynb \
    --container_list xqtl-pipeline/code/misc/xQTL_conda_packages.csv \
    --env bioinfo fastenloc leafcutter METAL methylation polyfun psichomics rna_quantification SOS stephenslab TensorQTL\
    --output some_image_name_prefix -n

In [ ]:
[global]
parameter: container_list = path
parameter: env = list
parameter: output = env
parameter: cwd=path("output")
parameter: def_file = "/mnt/mfs/hgrcgrid/homes/jl6702/git/xqtl-pipeline/code/misc/test.def"

In [ ]:
[contain_generator_1]
input: container_list, for_each="env"
output: f'{cwd}/{_env}/{_env}.yml'
python: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
    import os
    import yaml
    import pandas as pd
    container_list = pd.read_csv("$[_input]")
    container_tempt = dict()        
    ## dict
    container_list = container_list[container_list.Channel.notna()]
    container_tempt["name"]  = "$[_env]"
    container_tempt["channels"] = list(set(container_list.query("Environment.str.contains('$[_env]') | Environment == 'Universial'").Channel))
    container_tempt["dependencies"] = [ x.split("/")[-1] for x in  set(container_list.query("Environment.str.contains('$[_env]') | Environment == 'Universial'").query("Channel in @container_tempt['channels'] "  ).iloc[:,8] )]
    container_tempt["channels"].sort(reverse=True)
    container_tempt["channels"].append("nodefaults")
    container_tempt["channels"].sort(reverse=True)
    class IndentDumper(yaml.Dumper):
        def increase_indent(self, flow=False, indentless=False):
            return super(IndentDumper, self).increase_indent(flow, False)
    with open("$[_output]", 'w') as f:
        yaml.dump(container_tempt,f,sort_keys = False, Dumper=IndentDumper )

In [ ]:
[contain_generator_2]
output: f'{_input:na}.sif'
bash: expand = "$[ ]", stderr = f'{_output}.stderr', stdout = f'{_output}.stdout'
  cd $[_input:d]
  rm -f $[_output]
  singularity build --fakeroot $[_output] $[def_file]
  md5sum $[_output] > $[_output].md5